In [1]:
# python libraries
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML, display
# pydrake imports
from pydrake.all import (AddMultibodyPlantSceneGraph, DiagramBuilder,
                         Linearize, LinearQuadraticRegulator, LogVectorOutput,
                         MeshcatVisualizer, ModelVisualizer, Parser, Simulator, StartMeshcat,
                         MultibodyPlant, InitializeAutoDiffTuple, ExtractGradient, TrajectorySource,
                         FixedOffsetFrame, QuaternionFloatingJoint, UniformGravityFieldElement,
                         ExternallyAppliedSpatialForce, AbstractValue, ContactModel, GeometrySet,
                         CollisionFilterDeclaration, ContactResults, SpatialAcceleration, SpatialForce,
                         SceneGraph, DiagramBuilder_, AutoDiffXd)

from pydrake.common import temp_directory
from pydrake.geometry import (
    MeshcatVisualizerParams,
    Role,
)
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.systems.framework import BasicVector, LeafSystem
import pydrake.symbolic as sym
import pydrake.autodiffutils as autodiff
from pydrake.trajectories import PiecewisePolynomial

from underactuated import running_as_notebook

In [2]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()
meshcat.Set2dRenderMode()

INFO:drake:Meshcat listening for connections at https://067a6c9a-b930-4c28-8084-e86f86716a80.deepnoteproject.com/7000/


## Some testing things

In [3]:
# visualizer = ModelVisualizer(meshcat=meshcat)
# visualizer.parser().AddModels('./obstacle.urdf')
# visualizer.parser().AddModels('./obstacle_background.urdf')
# visualizer.parser().AddModels('./climber_model.urdf')
# visualizer.Run(loop_once=not running_as_notebook)
# meshcat.Delete()
# meshcat.DeleteAddedControls()

In [4]:
class TestInput(LeafSystem):
    def __init__(self):
        super().__init__()  # Don't forget to initialize the base class.
        self.DeclareVectorOutputPort(name="test", size=2, calc=self.DummyCalc)
        self.input = [0,0]

    def DummyCalc(self, context, output):
        output.SetFromVector(self.input)
    
    def setInput(self, new_input):
        self.input = new_input

## Hybrid Dynamics External Forces

In [5]:
class HybridClimberExternalForces(LeafSystem):
    def __init__(self, plant, sim_time_step=0.0001, disconnect_accel=0.1):
        super().__init__()  # Don't forget to initialize the base class.

        # [isConnected, num_disconnects, hand_accel]
        state = self.DeclareDiscreteState(3)
        self.DeclareStateOutputPort("y", state)  # One output: y=x.
        self.DeclarePeriodicDiscreteUpdateEvent(
            period_sec=0,  # timestep.
            offset_sec=0.0,  # The first event is at time zero.
            update=self.Update) # Call the Update method defined below.

        hand = plant.GetBodyByName("hand")
        self.hand = hand
        self.hand_index = hand.index()

        self.accels = self.DeclareAbstractInputPort("accels", AbstractValue.Make(
                                                        [SpatialAcceleration() for i in range(plant.num_bodies())]))
        self.contacts = self.DeclareAbstractInputPort("contact_results", AbstractValue.Make(ContactResults()))

        N_forces = 1
        self.DeclareAbstractOutputPort("spatial_forces", 
                                        lambda: AbstractValue.Make([ExternallyAppliedSpatialForce()
                                                                    for i in range(N_forces)]), 
                                        self.CalcSpatialForceContactForceCompensation,
                                        {self.all_state_ticket()})
     
    def Update(self, context, discrete_state):
        contact_results = self.contacts.Eval(context)
        body_accels = self.accels.Eval(context)

        hand_accel = body_accels[self.hand_index]
        discrete_state.get_mutable_vector().SetAtIndex(2, hand_accel)
        zdd_hand = hand_accel[5]

        # disconnect when the hand's acceleration in z is greater than disconnect_accel
        if (discrete_state.get_vector()[0] == 1 and zdd_hand > disconnect_accel 
            and discrete_state.get_vector()[1] == 0):
            discrete_state.get_mutable_vector().SetAtIndex(0, 0) # disconnect

            disconnects = discrete_state.get_vector()[1]
            discrete_state.get_mutable_vector().SetAtIndex(0, disconnects+1)

        # Reconnect the next time the robot touches the obstacle
        if (discrete_state.get_vector()[0] == 0 and 
            (contact_results.num_hydroelastic_contacts() + contact_results.num_point_pair_contacts()) > 0):
            discrete_state.get_mutable_vector().SetAtIndex(0, 1) # connect
    
    def CalcSpatialForceContactForceCompensation(self, context, output):
        hand_mass = self.hand.get_mass(context)
        hand_inertia = self.hand.GetSpatialInertiaInBodyFrame(context)

        hand_accel = discrete_state.get_vector()[2]

        tau = hand_accel[:3] * hand_inertia
        f = hand_accel[3:] * hand_mass

        force = SpatialForce(tau, f)
        if (discrete_state.get_vector()[0] == 1): # if disconnected
            force = SpatialForce([0, 0, 0, 0, 0, 0])

        output.set_value([ExternallyAppliedSpatialForce(hand.index(), force, [0,0,0])])

sim_time_step = 0.0001
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(
    builder, time_step=sim_time_step)
parser = Parser(plant)
parser.AddModels('./climber_model.urdf')
diagram = builder.Build()
x = HybridClimberExternalForces(plant)

## Setup system for iLQR dynamics

In [6]:
sim_time_step = 0.0001
builder = DiagramBuilder()
# plant = MultibodyPlant(0)
# builder.AddSystem(plant)
plant, scene_graph = AddMultibodyPlantSceneGraph(
    builder, time_step=0)
parser = Parser(plant)

# Loading models.
# Load the climber 
parser.AddModels('./climber_model.urdf')

# Load the obstacle
parser.AddModels('./obstacle.urdf')
parser.AddModels('./obstacle_background.urdf')

# Weld the obstacle to the world so that it's fixed during the simulation.
obstacle_frame = plant.GetFrameByName("obstacle_base")
plant.WeldFrames(plant.world_frame(), obstacle_frame)
obstacle_background_frame = plant.GetFrameByName("obstacle_base_background")
plant.WeldFrames(plant.world_frame(), obstacle_background_frame)

# Finalize the plant after loading the scene.
plant.Finalize()

plant_context = plant.CreateDefaultContext()

# Set the initial pose for the robot climber
hand = plant.GetBodyByName("hand")
X_WorldObstacle = obstacle_frame.CalcPoseInWorld(plant_context)
X_ObstacleHand = RigidTransform(
    RollPitchYaw(np.asarray([0, 0, 0]) * np.pi / 180), p=[0.03,0,0.1])
X_WorldHand = X_WorldObstacle.multiply(X_ObstacleHand)
plant.SetDefaultFreeBodyPose(hand, X_WorldHand)

plant.get_actuation_input_port().FixValue(plant_context, [0,0])

# Add external force for hybrid model
# PROBLEM: THIS BEING DISCRETE IS BAD
# hybrid_force = builder.AddSystem(HybridClimberExternalForces(plant))
# builder.Connect(hybrid_force.get_output_port(1), plant.get_applied_spatial_force_input_port())
# builder.Connect(plant.get_body_spatial_accelerations_output_port(), hybrid_force.get_input_port(0))
# builder.Connect(plant.get_contact_results_output_port(), hybrid_force.get_input_port(1))

# controller = builder.AddSystem(TestInput())
# builder.Connect(controller.get_output_port(0),
#                 plant.get_actuation_input_port())

diagram = builder.Build()

root_context = diagram.CreateDefaultContext()
plant_context = plant.GetMyMutableContextFromRoot(root_context)
scene_graph_context = scene_graph.GetMyMutableContextFromRoot(root_context)

# Disable all collisions
inspector = scene_graph.model_inspector()
filter_manager = scene_graph.collision_filter_manager(scene_graph_context)
geometry_set = GeometrySet(inspector.GetAllGeometryIds())
filter_manager.Apply(CollisionFilterDeclaration().ExcludeWithin(geometry_set))

num_states = plant.num_multibody_states() 
num_inputs = plant.get_actuation_input_port().size()
print(num_states, num_inputs)
print(plant.get_applied_spatial_force_input_port().size())

19 2
0


In [7]:
# # # TESTING STUFF BELOW DELETE LATER
# hand = plant.GetBodyByName("hand")
# hand_accel = hand.EvalSpatialAccelerationInWorld(plant_context)
# contact_results = plant.get_contact_results_output_port().Eval(plant_context)
# print((contact_results.num_hydroelastic_contacts() + contact_results.num_point_pair_contacts()))

# print(plant_context.get_continuous_state_vector())
plant.get_actuation_input_port().FixValue(plant_context, [0,0])
time_derivs = plant.EvalTimeDerivatives(plant_context).get_vector()
print(time_derivs)
# %tb

## iLQR

In [8]:
def f_derivs(plant, scene_graph, context, root_context, x, u):
    num_states = plant.num_multibody_states() 
    num_inputs = plant.get_actuation_input_port().size()

    setPlantPositionsAndVelocities(plant, context, x)
    setInput(plant, context, u)

    builder_autodiff = DiagramBuilder_[AutoDiffXd]()

    plant_autodiff = plant.ToAutoDiffXd()
    scene_graph_autodiff = scene_graph.ToAutoDiffXd()

    builder_autodiff.AddSystem(plant_autodiff)
    builder_autodiff.AddSystem(scene_graph_autodiff)

    builder_autodiff.Connect(scene_graph_autodiff.get_query_output_port(), 
                            plant_autodiff.get_geometry_query_input_port())
    builder_autodiff.Connect(plant_autodiff.get_geometry_poses_output_port(), 
                            scene_graph_autodiff.get_source_pose_port(plant_autodiff.get_source_id()))

    diagram_autodiff = builder_autodiff.Build()

    root_context_autodiff = diagram_autodiff.CreateDefaultContext()
    context_autodiff = plant_autodiff.GetMyMutableContextFromRoot(root_context_autodiff);
    root_context_autodiff.SetTimeStateAndParametersFrom(root_context);

    x0 = context.get_continuous_state_vector().CopyToVector()
    u0 = plant.get_actuation_input_port().Eval(context)

    autodiff_args = InitializeAutoDiffTuple(x0, u0)

    # Set state and input in autodiff
    plant_autodiff.get_actuation_input_port().FixValue(context_autodiff, autodiff_args[1])
    context_autodiff.get_mutable_continuous_state_vector().SetFromVector(autodiff_args[0])

    # Get full jacobians
    autodiff_xdot = plant_autodiff.AllocateTimeDerivatives();
    plant_autodiff.CalcTimeDerivatives(context_autodiff, autodiff_xdot);
    autodiff_xdot_vec = autodiff_xdot.CopyToVector()
    AB = ExtractGradient(autodiff_xdot_vec)
    A = AB[:, :num_states]
    B = AB[:, -num_inputs:]

    # truncate to state and input vectors
    # states = [qw=1, qx=0, qy=0, qz=0, x, y, z, theta0, theta1, theta2, 
    #           w_hand_x, w_hand_y, w_hand_z, v_hand_x, v_hand_y, v_hand_z, theta0d, theta1d, theta2d]
    # x = [x_hand, z_hand, theta0, theta1, theta2, xd_hand, zd_hand, theta0d, theta1d, theta2d]
    # u = [theta1dd, theta2dd]

    x_indices = [4,6,7,8,9,13,15,16,17,18]
    f_x = np.array(A)[x_indices, :][:,x_indices]
    f_u = np.array(B)[x_indices, :]

    print("f_x, f_u", f_x, f_u)

    return f_x, f_u

In [9]:
def setPlantPositionsAndVelocities(plant, context, x):
    # positions = [qw=1, qx=0, qy=0, qz=0, x, y, z, theta0, theta1, theta2]
    plant.SetPositions(context, np.array([1, 0, 0, 0, x[0], 0, 
                                x[1], x[2], x[3], x[4]]))
    
    # velocities = [w_hand_x, w_hand_y, w_hand_z, v_hand_x, v_hand_y, v_hand_z, theta0d, theta1d, theta2d]
    plant.SetVelocities(context, np.array([0, 0, 0, x[5], 0, 
                                x[6], x[7], x[8], x[9]]))

def setInput(plant, context, u):  
    plant.get_actuation_input_port().FixValue(context, u)

In [10]:
n_x = 10 
n_u = 2
def continuous_dynamics(plant, context, x, u):
    # x = [x_hand, z_hand, theta0, theta1, theta2, xd_hand, zd_hand, theta0d, theta1d, theta2d]
    # u = [theta1dd, theta2dd]

    setPlantPositionsAndVelocities(plant, context, x)
    plant.get_actuation_input_port().FixValue(context, u)

    # hybrid joint
    hand = plant.GetBodyByName("hand")
    hand_accel = hand.EvalSpatialAccelerationInWorld(context)
    contact_results = plant.get_contact_results_output_port().Eval(context)
    num_contacts = contact_results.num_hydroelastic_contacts() + contact_results.num_point_pair_contacts()
    print("num_contacts", num_contacts)
    print("hand_accel", hand_accel)

    # d[qw, qx, qy, qz, x, y, z, theta0, theta1, theta2, w_hand_x, w_hand_y, w_hand_z, v_hand_x, 
    # v_hand_y, v_hand_z, theta0d, theta1d, theta2d]/dt
    time_derivs = plant.EvalTimeDerivatives(context).get_vector()

    # isolate the entries in time_derivs we care about
    # print(time_derivs)
    xd = np.array([time_derivs[4], time_derivs[6], time_derivs[7], time_derivs[8], time_derivs[9],
                time_derivs[13], time_derivs[15], time_derivs[16], time_derivs[17], time_derivs[18]])
    print("xd", xd)
    return xd

In [11]:
dt = 0.1
def discrete_dynamics(plant, context, x, u):
    x_next = x + dt * continuous_dynamics(plant, context, x, u)
    return x_next

In [12]:
def rollout(x0, u_trj, plant, context):
    x_trj = np.zeros((u_trj.shape[0] + 1, x0.shape[0]))
    x_trj[0] = x0
    for i in range(len(u_trj)):
        x_trj[i+1] = discrete_dynamics(plant, context, x_trj[i], u_trj[i])

    return x_trj

In [13]:
x_target = 0
z_target = 0.6

def cost_stage(x, u):
    m = sym if x.dtype == object else np  # Check type for autodiff
    c_position = (x[0] - 0.03) ** 2 + (x[1] - 0.6) ** 2
    c_speed = x[5] ** 2 + x[6] ** 2
    c_control = (u[0] ** 2 + u[1] ** 2) * 0.1
    return c_position + c_speed + c_control


def cost_final(x):
    m = sym if x.dtype == object else np  # Check type for autodiff
    c_position = (x[0] - 0.03) ** 2 + (x[1] - 0.6) ** 2
    c_speed = x[5] ** 2 + x[6] ** 2
    return c_position + c_speed

In [14]:
def cost_trj(x_trj, u_trj):
    total = 0.0
    for i in range(len(u_trj)):
        total += cost_stage(x_trj[i], u_trj[i])
    total += cost_final(x_trj[-1])
    return total

In [15]:
n_x = 10 # [x_hand, z_hand, theta0, theta1, theta2, xd_hand, zd_hand, theta0d, theta1d, theta2d]
n_u = 2 # [theta1dd, theta2dd]
class derivatives:
    def __init__(self, plant, scene_graph, context, root_context, discrete_dynamics, cost_stage, cost_final, n_x, n_u):
        self.x_sym = np.array(
            [sym.Variable("x_{}".format(i)) for i in range(n_x)]
        )
        self.u_sym = np.array(
            [sym.Variable("u_{}".format(i)) for i in range(n_u)]
        )
        x = self.x_sym
        u = self.u_sym
        self.plant = plant
        self.context = context
        self.scene_graph = scene_graph
        self.root_context = root_context

        l = cost_stage(x, u)
        self.l_x = sym.Jacobian([l], x).ravel()
        self.l_u = sym.Jacobian([l], u).ravel()
        self.l_xx = sym.Jacobian(self.l_x, x)
        self.l_ux = sym.Jacobian(self.l_u, x)
        self.l_uu = sym.Jacobian(self.l_u, u)

        l_final = cost_final(x)
        self.l_final_x = sym.Jacobian([l_final], x).ravel()
        self.l_final_xx = sym.Jacobian(self.l_final_x, x)

    def stage(self, x, u):
        env = {self.x_sym[i]: x[i] for i in range(x.shape[0])}
        env.update({self.u_sym[i]: u[i] for i in range(u.shape[0])})

        l_x = sym.Evaluate(self.l_x, env).ravel()
        l_u = sym.Evaluate(self.l_u, env).ravel()
        l_xx = sym.Evaluate(self.l_xx, env)
        l_ux = sym.Evaluate(self.l_ux, env)
        l_uu = sym.Evaluate(self.l_uu, env)

        f_x, f_u = f_derivs(self.plant, self.scene_graph, self.context, self.root_context, x, u)

        return l_x, l_u, l_xx, l_ux, l_uu, f_x, f_u

    def final(self, x):
        env = {self.x_sym[i]: x[i] for i in range(x.shape[0])}

        l_final_x = sym.Evaluate(self.l_final_x, env).ravel()
        l_final_xx = sym.Evaluate(self.l_final_xx, env)

        return l_final_x, l_final_xx


derivs = derivatives(plant, scene_graph, plant_context, root_context, discrete_dynamics, cost_stage, cost_final, n_x, n_u)
# Test the output:

# x = [x_hand, z_hand, theta0, theta1, theta2, xd_hand, zd_hand, theta0d, theta1d, theta2d]
# u = [theta1dd, theta2dd]
# x = np.array([0.03, 0.1, 0, 0, 0, 0, 0, 0, 0, 0])
# u = np.array([0, 0])
# print(derivs.stage(x, u))
# print(derivs.final(x))

In [16]:
def Q_terms(l_x, l_u, l_xx, l_ux, l_uu, f_x, f_u, V_x, V_xx):
    Q_x = l_x + (V_x.T.dot(f_x)).T
    Q_u = l_u + (V_x.T.dot(f_u)).T
    Q_xx = l_xx + f_x.T.dot(V_xx).dot(f_x)
    Q_ux = l_ux + f_u.T.dot(V_xx).dot(f_x)
    Q_uu = l_uu + f_u.T.dot(V_xx).dot(f_u)
    return Q_x, Q_u, Q_xx, Q_ux, Q_uu

In [17]:
def gains(Q_uu, Q_u, Q_ux):
    Q_uu_inv = np.linalg.inv(Q_uu)
    k = -Q_uu_inv @ Q_u
    K = -Q_uu_inv @ Q_ux
    return k, K

In [18]:
def V_terms(Q_x, Q_u, Q_xx, Q_ux, Q_uu, K, k):
    V_x = Q_x + (Q_u.T.dot(K)).T + 1/2 * ((k.T.dot(Q_ux)).T + Q_ux.T.dot(k) + (k.T.dot(Q_uu).dot(K)).T + K.T.dot(Q_uu).dot(k))
    V_xx = Q_xx + K.T.dot(Q_ux) + Q_ux.T.dot(K) + K.T.dot(Q_uu).dot(K)
    return V_x, V_xx

In [19]:
def expected_cost_reduction(Q_u, Q_uu, k):
    return -Q_u.T.dot(k) - 0.5 * k.T.dot(Q_uu.dot(k))

In [20]:
def forward_pass(x_trj, u_trj, k_trj, K_trj, plant, context):
    x_trj_new = np.zeros(x_trj.shape)
    x_trj_new[0, :] = x_trj[0, :]
    u_trj_new = np.zeros(u_trj.shape)
    
    for n in range(u_trj.shape[0]):
        u_trj_new[n,:] = u_trj[n] + k_trj[n] + K_trj[n] @ (x_trj_new[n] - x_trj[n])# Apply feedback law
        x_trj_new[n+1,:] = discrete_dynamics(plant, context, x_trj_new[n], u_trj_new[n]) # Apply dynamics
    return x_trj_new, u_trj_new

In [21]:
def backward_pass(x_trj, u_trj, regu):
    k_trj = np.zeros([u_trj.shape[0], u_trj.shape[1]])
    K_trj = np.zeros([u_trj.shape[0], u_trj.shape[1], x_trj.shape[1]])
    expected_cost_redu = 0
    # Set terminal boundary condition here (V_x, V_xx)
    V_x, V_xx = derivs.final(x_trj[-1])
    for n in range(u_trj.shape[0] - 1, -1, -1):
        # First compute derivatives, then the Q-terms
        l_x, l_u, l_xx, l_ux, l_uu, f_x, f_u = derivs.stage(x_trj[n], u_trj[n])
        Q_x, Q_u, Q_xx, Q_ux, Q_uu = Q_terms(l_x, l_u, l_xx, l_ux, l_uu, f_x, f_u, V_x, V_xx)
        # We add regularization to ensure that Q_uu is invertible and nicely conditioned
        Q_uu_regu = Q_uu + np.eye(Q_uu.shape[0]) * regu
        k, K = gains(Q_uu_regu, Q_u, Q_ux)
        k_trj[n, :] = k
        K_trj[n, :, :] = K
        V_x, V_xx = V_terms(Q_x, Q_u, Q_xx, Q_ux, Q_uu, K, k)
        expected_cost_redu += expected_cost_reduction(Q_u, Q_uu, k)
    return k_trj, K_trj, expected_cost_redu

In [22]:
def run_ilqr(plant, context, x0, N, max_iter=50, regu_init=100):
    # First forward rollout
    u_trj = np.random.randn(N - 1, n_u) * 0.0001
    x_trj = rollout(x0, u_trj, plant, plant_context)
    total_cost = cost_trj(x_trj, u_trj)
    regu = regu_init
    max_regu = 10000
    min_regu = 0.01

    # Setup traces
    cost_trace = [total_cost]
    expected_cost_redu_trace = []
    redu_ratio_trace = [1]
    redu_trace = []
    regu_trace = [regu]

    # Run main loop
    for it in range(max_iter):
        # Backward and forward pass
        k_trj, K_trj, expected_cost_redu = backward_pass(x_trj, u_trj, regu)
        x_trj_new, u_trj_new = forward_pass(x_trj, u_trj, k_trj, K_trj, plant, context)
        # Evaluate new trajectory
        total_cost = cost_trj(x_trj_new, u_trj_new)
        cost_redu = cost_trace[-1] - total_cost
        redu_ratio = cost_redu / abs(expected_cost_redu)
        # Accept or reject iteration
        if cost_redu > 0:
            # Improvement! Accept new trajectories and lower regularization
            redu_ratio_trace.append(redu_ratio)
            cost_trace.append(total_cost)
            x_trj = x_trj_new
            u_trj = u_trj_new
            regu *= 0.7
        else:
            # Reject new trajectories and increase regularization
            regu *= 2.0
            cost_trace.append(cost_trace[-1])
            redu_ratio_trace.append(0)
        regu = min(max(regu, min_regu), max_regu)
        regu_trace.append(regu)
        redu_trace.append(cost_redu)

        # Early termination if expected improvement is small
        if expected_cost_redu <= 1e-6:
            break

    return x_trj, u_trj, cost_trace, regu_trace, redu_ratio_trace, redu_trace

# Setup problem and call iLQR
# x = [x_hand, z_hand, theta0, theta1, theta2, xd_hand, zd_hand, theta0d, theta1d, theta2d]
x0 = np.array([0.03, 0.1, 1, 2, 0, 0, 0, 0, 0, 0])
N = 50
max_iter = 50
regu_init = 100
x_trj, u_trj, cost_trace, regu_trace, redu_ratio_trace, redu_trace = run_ilqr(
    plant, plant_context, x0, N, max_iter, regu_init
)

print(x_trj)
print(u_trj)

# Turn result into TrajectorySource
breaks = np.array([dt * t for t in range(len(u_trj))])
piecewise_u_traj = PiecewisePolynomial.FirstOrderHold(breaks, u_trj.T)

traj_source = TrajectorySource(piecewise_u_traj)

num_contacts 1
hand_accel SpatialAcceleration(
  alpha=[0.0, 2.81846667302448e-13, 0.0],
  a=[-26.922701816644032, 0.0, -16.500264609792815],
)
xd [   0.            0.            0.            0.            0.
  -26.92270182  -16.50026461   30.58945267 -106.59886419   94.65116171]
num_contacts 0
hand_accel SpatialAcceleration(
  alpha=[0.0, -1.893679468810204e-12, 0.0],
  a=[-2.11349660037307, 0.0, -13.354467215253655],
)
xd [ -2.69227018  -1.65002646   3.05894527 -10.65988642   9.46511617
  -2.1134966  -13.35446722  50.29593492 -32.15420694 -22.58904449]
num_contacts 0
hand_accel SpatialAcceleration(
  alpha=[0.0, -3.896756293628615e-12, 0.0],
  a=[12.498906729255745, 0.0, -16.898875832339165],
)
xd [ -2.90361984  -2.98547318   8.08853876 -13.87530711   7.20621172
  12.49890673 -16.89887583  43.5297111  -91.98588619  36.69124038]
num_contacts 0
hand_accel SpatialAcceleration(
  alpha=[0.0, 0.0, 0.0],
  a=[31.52677505980538, 0.0, 18.568734461090827],
)
xd [  -1.65372917   -4.67536077  

RuntimeError: NaN is detected in the initialization of an environment.

In [0]:
plt.subplots(figsize=(10, 6))
# Plot results
plt.subplot(2, 2, 1)
plt.plot(cost_trace)
plt.xlabel("# Iteration")
plt.ylabel("Total cost")
plt.title("Cost trace")

plt.subplot(2, 2, 2)
delta_opt = np.array(cost_trace) - cost_trace[-1]
plt.plot(delta_opt)
plt.yscale("log")
plt.xlabel("# Iteration")
plt.ylabel("Optimality gap")
plt.title("Convergence plot")

plt.subplot(2, 2, 3)
plt.plot(redu_ratio_trace)
plt.title("Ratio of actual reduction and expected reduction")
plt.ylabel("Reduction ratio")
plt.xlabel("# Iteration")

plt.subplot(2, 2, 4)
plt.plot(regu_trace)
plt.title("Regularization trace")
plt.ylabel("Regularization")
plt.xlabel("# Iteration")
plt.tight_layout()

## Simulate

In [0]:
def create_scene(sim_time_step=0.0001):
    # Clean up the Meshcat instance.
    meshcat.Delete()
    meshcat.DeleteAddedControls()

    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(
        builder, time_step=sim_time_step)
    parser = Parser(plant)

    # Loading models.
    # Load the climber 
    parser.AddModels('./climber_model.urdf')
    # hand_frame = plant.GetFrameByName("hand")
    # plant.WeldFrames(plant.world_frame(), hand_frame)

    # Load the obstacle
    parser.AddModels('./obstacle.urdf')
    parser.AddModels('./obstacle_background.urdf')

    #Weld the obstacle to the world so that it's fixed during the simulation.
    obstacle_frame = plant.GetFrameByName("obstacle_base")
    plant.WeldFrames(plant.world_frame(), obstacle_frame)
    obstacle_background_frame = plant.GetFrameByName("obstacle_base_background")
    plant.WeldFrames(plant.world_frame(), obstacle_background_frame)
    
    X_JointWorld = RigidTransform(RollPitchYaw(np.asarray([0, 0, 0]) * np.pi / 180), p=[0.024,0,-0.2])
    handFrame = plant.GetFrameByName("hand")
    jointWorldFrame = FixedOffsetFrame("jointWorld", plant.world_frame(), X_JointWorld)
    plant.AddFrame(jointWorldFrame)
    floatingBaseJoint = QuaternionFloatingJoint("floating_hand_joint", jointWorldFrame, handFrame)
    plant.AddJoint(floatingBaseJoint)

    # Finalize the plant after loading the scene.
    plant.Finalize()
    # We use the default context to calculate the transformation of the table
    # in world frame but this is NOT the context the Diagram consumes.
    plant_context = plant.CreateDefaultContext()

    print(plant.num_positions())
    print(plant.num_velocities())

    # Set the initial pose for the robot climber
    # hand = plant.GetBodyByName("hand")
    # X_WorldObstacle = obstacle_frame.CalcPoseInWorld(plant_context)
    # X_ObstacleHand = RigidTransform(
    #     RollPitchYaw(np.asarray([0, 0, 0]) * np.pi / 180), p=[0.03,0,0.1])
    # X_WorldHand = X_WorldObstacle.multiply(X_ObstacleHand)
    # plant.SetDefaultFreeBodyPose(hand, X_WorldHand)

    floatingBaseJoint.Lock(plant_context)
    
    controller = builder.AddSystem(TestInput())
    builder.Connect(controller.get_output_port(0),
                    plant.get_actuation_input_port())

    # builder.AddSystem(traj_source)
    # builder.Connect(traj_source.get_output_port(0),
    #                 plant.get_actuation_input_port())

    # Add visualizer to visualize the geometries.
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat,
        MeshcatVisualizerParams(role=Role.kPerception, prefix="visual"))

    diagram = builder.Build()
    return diagram, visualizer

In [0]:
def initialize_simulation(diagram):
    simulator = Simulator(diagram)
    simulator.Initialize()
    simulator.set_target_realtime_rate(1.)
    return simulator

def run_simulation(sim_time_step):
    diagram, visualizer = create_scene(sim_time_step)
    simulator = initialize_simulation(diagram)
    visualizer.StartRecording()
    simulator.AdvanceTo(5.0)
    visualizer.PublishRecording()

# Run the simulation with a small time step. Try gradually increasing it!
run_simulation(sim_time_step=0.0001)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=067a6c9a-b930-4c28-8084-e86f86716a80' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>